# Meta Analysis for protein corona

In [ ]:
############################################################
# Random Effects Meta-Analysis of Protein Corona Detection
# Author: Alexa Canchola
# Advisor: Wei-Chun Chou
# Date: Jul 22, 2025
############################################################

In [ ]:
#### 1.  Libraries  --------------------------------------------------------
if (!requireNamespace("pacman", quietly = TRUE))
  install.packages("pacman", repos = "https://cloud.r-project.org")

pacman::p_load(
  readxl, dplyr, tidyr, stringr, metafor,
  ggplot2, scales, knitr, kableExtra
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘svglite’



kableExtra installed



In [ ]:
#### 2.  Import & minimal cleaning  ----------------------------------------
data_path <- "PC-DB_for_Meta-Analysis - v2.xlsx"
raw <- read_excel(data_path, sheet = 1)

df <- raw %>%
  transmute(
    study_id = `Study ID`,
    np_id    = `NP Entry ID`,
    material = `NP Sub-Type`,
    size_nm  = as.numeric(`Size (nm)`),
    zeta_mV  = as.numeric(`Zeta Potential (mV)`),

    # protein intensity columns (keep raw names from sheet)
    APOE,
    APOB100 = APOB,
    C3      = CO3,
    CLU     = CLUS
  )



Rows: 597
Columns: 10
$ `NP Entry ID`         <chr> "A001", "A002", "A003", "A004", "A005", "A006", …
$ `Study ID`            <chr> "A01", "A01", "A01", "A01", "A01", "A01", "A02",…
$ `Modification Type`   <chr> "No Modification", "No Modification", "Amines", …
$ `NP Sub-Type`         <chr> "Silica", "Silica", "Silica", "Silica", "Silica"…
$ `Size (nm)`           <chr> "101", "165", "100", "192", "105", "203", "193.4…
$ `Zeta Potential (mV)` <chr> "-15", "-12", "-11", "-7", "-15", "-13", "39.9",…
$ APOE                  <dbl> 5.1929806, 2.9713805, 3.0479906, 3.3245025, 3.96…
$ APOB                  <chr> "1.211149915", "1.356342052", "0.956093828999999…
$ CO3                   <chr> "1.0944008489999999", "1.1376095900000001", "1.2…
$ CLUS                  <dbl> 0.2438552, 0.4465021, 0.2269439, 0.5026110, 0.19…


In [ ]:
# convert intensities → 0 / 1 detection flags
prot_cols <- c("APOE", "APOB100", "C3", "CLU")
detectify  <- function(v) as.integer(!is.na(v) & v != 0 & v != "ND")
df <- df %>% mutate(across(all_of(prot_cols), detectify))

# derive material / size / charge strata
df <- df %>%
  mutate(
    mat_grp = case_when(
      str_detect(material, regex("silica", TRUE))        ~ "Silica",
      str_detect(material, regex("metal|oxide", TRUE))   ~ "Metal/Metal-oxide",
      TRUE                                               ~ "Lipid / Polymer"
    ),
    size_bin   = if_else(size_nm < 100, "<100 nm", "≥100 nm"),
    charge_bin = cut(
      zeta_mV,
      breaks  = c(-Inf, -20, 0, Inf),
      labels  = c("≤−20 mV", "−20–0 mV", ">0 mV")
    )
  )

In [ ]:
#### 3.  Long format (one NP × protein per row)  ---------------------------
df_long <- df %>%
  pivot_longer(cols = all_of(prot_cols),
               names_to = "protein",
               values_to = "detect")

In [ ]:
#### 4.  Helper: random-effects pooled proportion  -------------------------
meta_prop_pool <- function(data) {
  agg <- data %>%
    group_by(study_id) %>%
    summarise(x = sum(detect, na.rm = TRUE),
              n = n(),
              .groups = "drop") %>%
    filter(n > 0)

  if (nrow(agg) < 2) return(NULL)

  esc <- escalc(measure = "PLO", xi = x, ni = n, data = agg)
  fit <- rma(yi, vi, method = "REML", data = esc)

  tibble(
    k       = fit$k,
    N       = sum(agg$n),
    prop    = transf.ilogit(fit$b),
    ci_low  = transf.ilogit(fit$ci.lb),
    ci_high = transf.ilogit(fit$ci.ub),
    I2      = fit$I2
  )
}

In [ ]:
#### 5.  Define analysis strata  -------------------------------------------
strata_def <- tribble(
  ~mat_grp,              ~size_bin,  ~label,                       ~is_ref,
  "Silica",              "<100 nm",  "Silica <100 nm",             TRUE,
  "Silica",              "≥100 nm",  "Silica ≥100 nm",             FALSE,
  "Metal/Metal-oxide",   "<100 nm",  "Metal/Metal-oxide <100 nm",  FALSE,
  "Metal/Metal-oxide",   "≥100 nm",  "Metal/Metal-oxide ≥100 nm",  FALSE,
  "Lipid / Polymer",     "<100 nm",  "Lipid / Polymer <100 nm",    FALSE,
  "Lipid / Polymer",     "≥100 nm",  "Lipid / Polymer ≥100 nm",    FALSE
)

In [ ]:
#### 6.  Loop over proteins to build main table  ---------------------------
results <- purrr::map(prot_cols, function(prot) {
  part <- df_long %>% filter(protein == prot)

  strata_def %>%
    rowwise() %>%
    mutate(
      pool = list(
        meta_prop_pool(
          part %>%
            filter(mat_grp  == .env$mat_grp,
                   size_bin == .env$size_bin)
        )
      )
    ) %>%
    unnest(pool) %>%
    mutate(protein = prot)
}) %>% bind_rows() %>% relocate(protein)


In [ ]:
#### 7.  Fixed meta-regression contrasts  ----------------------------------
get_contrast_p <- function(df_long, prot, ref_label, strata_df) {

  # keep only rows for this protein
  d <- df_long %>% filter(protein == prot)

  # tag each NP row with its stratum label
  d_tagged <- purrr::pmap_dfr(
    strata_df,
    function(mat_grp, size_bin, label, ...) {
      d %>%
        filter(mat_grp == !!mat_grp, size_bin == !!size_bin) %>%
        mutate(stratum = label)
    }
  )

  if (nrow(d_tagged) == 0)
    return(tibble(label = strata_df$label, p_val = NA_real_))

  # aggregate study × stratum
  agg <- d_tagged %>%
    group_by(study_id, stratum) %>%
    summarise(x = sum(detect, na.rm = TRUE),
              n = n(), .groups = "drop") %>%
    filter(n > 0)

  if (length(unique(agg$stratum)) < 2)
    return(tibble(label = strata_df$label, p_val = NA_real_))

  # meta-regression
  agg$stratum <- factor(agg$stratum, levels = strata_df$label)
  agg$stratum <- relevel(agg$stratum, ref = ref_label)
  esc <- escalc(measure = "PLO", xi = x, ni = n, data = agg)
  fit <- rma(yi, vi, mods = ~ stratum, data = esc, method = "REML")

  tibble(
    label = strata_df$label[-1],          # exclude intercept
    p_val = summary(fit)$pval[-1]
  )
}

# compute p-values for each protein
pvals <- purrr::imap_dfr(
  split(results, results$protein),
  function(tbl, prot) {
    ref_lbl <- tbl$label[tbl$is_ref][1]
    get_contrast_p(df_long, prot, ref_lbl, strata_def) %>%
      mutate(protein = prot)
  }
)

# merge & format p column
tableX <- results %>%
  left_join(pvals, by = c("protein", "label")) %>%
  mutate(`p vs ref*` = case_when(
    is_ref        ~ "—ref—",
    is.na(p_val)  ~ "n.e.",
    p_val < 0.001 ~ "<0.001",
    TRUE          ~ formatC(p_val, digits = 3, format = "f")
  )) %>%
  select(protein, label, k, N,
         prop, ci_low, ci_high, I2, `p vs ref*`)

In [ ]:
#### 8.  Display table  ----------------------------------------------------
 kable(tableX, digits = 2)



|protein |label                     |  k|   N|      prop| ci_low| ci_high|    I2|p vs ref* |
|:-------|:-------------------------|--:|---:|---------:|------:|-------:|-----:|:---------|
|APOE    |Silica <100 nm            |  5|  89| 0.9461682|   0.83|    0.98|  0.00|—ref—     |
|APOE    |Silica ≥100 nm            |  8|  71| 0.8928365|   0.74|    0.96|  0.00|0.461     |
|APOE    |Metal/Metal-oxide <100 nm | 10| 189| 0.8234829|   0.56|    0.94| 77.03|0.090     |
|APOE    |Metal/Metal-oxide ≥100 nm |  8|  35| 0.8159427|   0.61|    0.93|  3.75|0.189     |
|APOE    |Lipid / Polymer <100 nm   | 13|  60| 0.8273580|   0.66|    0.92| 13.86|0.168     |
|APOE    |Lipid / Polymer ≥100 nm   | 23| 145| 0.8344719|   0.74|    0.90|  9.05|0.163     |
|APOB100 |Silica <100 nm            |  5|  89| 0.8045237|   0.70|    0.88|  0.00|—ref—     |
|APOB100 |Silica ≥100 nm            |  8|  71| 0.8671047|   0.69|    0.95|  2.15|0.816     |
|APOB100 |Metal/Metal-oxide <100 nm | 10| 189| 0.8022990|   0.53|   

In [ ]:
write.csv(tableX, "meta_analysis_results.csv", row.names = FALSE)


In [ ]:
# 1) how many entries survive each step
n_raw <- nrow(df)                         # 598

n_complete <- df %>%
  filter(!is.na(size_nm), !is.na(material)) %>% nrow()

n_in_strata <- df %>%
  filter(!is.na(size_nm), !is.na(material)) %>%
  mutate(mat_grp = case_when(
    str_detect(material, regex("silica", TRUE))            ~ "Silica",
    str_detect(material, regex("metal|oxide", TRUE))       ~ "Metal/Metal-oxide",
    TRUE                                                   ~ "Lipid / Polymer"
  )) %>%
  filter(mat_grp != "Other") %>%  nrow()

cat("Raw file:", n_raw,
    "\nAfter complete vars:", n_complete,
    "\nAfter strata filters:", n_in_strata, "\n")

Raw file: 597 
After complete vars: 589 
After strata filters: 589 


In [ ]:
## 8 rows missing size or material
df %>% filter(is.na(size_nm) | is.na(material))


study_id,np_id,material,size_nm,zeta_mV,APOE,APOB100,C3,CLU,mat_grp,size_bin,charge_bin
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>
A44,A407,Silica,NA,-13.4,1,1,1,1,Silica,NA,−20–0 mV
A44,A408,Silica,NA,-15.6,1,1,1,1,Silica,NA,−20–0 mV
A44,A409,Silica,NA,-17.8,1,1,1,1,Silica,NA,−20–0 mV
A44,A410,Silica,NA,-26.8,1,1,1,1,Silica,NA,≤−20 mV
A44,A411,PEG,NA,-37.8,1,1,1,1,Lipid / Polymer,NA,≤−20 mV
A44,A412,PEG,NA,-37.8,1,1,1,1,Lipid / Polymer,NA,≤−20 mV
A44,A413,PEG,NA,-38.6,1,1,1,1,Lipid / Polymer,NA,≤−20 mV
A44,A414,PEG,NA,-36.4,1,1,1,1,Lipid / Polymer,NA,≤−20 mV
